##**Importing Required Packages**


In [ ]:
!pip install mido

     |████████████████████████████████| 51 kB 2.1 MB/s 


In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

##**Load MID file/files**

In [ ]:
!unzip /content/chillhopdata.zip -d chillhop

In [ ]:
import os

notes = []
for song in os.listdir("/content/chillhop"):
  mid = MidiFile('/content/chillhop/' + song)
  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type =='note_on':
      data = msg.bytes()
      notes.append(data[1])
  

##**Scale Data**

In [ ]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1, 1)))

##**Create Train Data**

In [ ]:
notes = [list(note) for note in notes]

X = []
y = []

n_prev=30

for i in range(len(notes) - n_prev):
  X.append(notes[i:i+n_prev])
  y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y = y[:-300]

##**Build LSTM**

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
model.summary()

optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 256)           264192    
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 256)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 128)           197120    
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##**Training**

In [ ]:
model.fit(np.array(X), np.array(y), batch_size=16, epochs=10, verbose=1)

Epoch 1/10
195/195 [==============================] - 27s 139ms/step - loss: 0.0179
Epoch 2/10
195/195 [==============================] - 27s 139ms/step - loss: 0.0170
Epoch 3/10
195/195 [==============================] - 27s 140ms/step - loss: 0.0161
Epoch 4/10
195/195 [==============================] - 28s 144ms/step - loss: 0.0162
Epoch 5/10
195/195 [==============================] - 28s 144ms/step - loss: 0.0161
Epoch 6/10
195/195 [==============================] - 27s 139ms/step - loss: 0.0158
Epoch 7/10
195/195 [==============================] - 28s 146ms/step - loss: 0.0153
Epoch 8/10
195/195 [==============================] - 28s 144ms/step - loss: 0.0153
Epoch 9/10
195/195 [==============================] - 28s 143ms/step - loss: 0.0152
Epoch 10/10
195/195 [==============================] - 28s 145ms/step - loss: 0.0152


##**Generating & Saving LSTM Music**

In [ ]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')